## Install cobrapy

In [5]:
%pip install cobra  --quiet

## Import C4 model from Github

In [4]:
# Retrieving the model
!wget -q https://raw.githubusercontent.com/sshameer/DielLeafModeling/refs/heads/main/C4_model.sbml

## Import functions required to read sbml, write sbml and perform pFBA

In [6]:
from cobra.io import read_sbml_model
from cobra.io import write_sbml_model
from cobra.flux_analysis import pfba

## Import the model and constrain diel leaf model for C4 photosynthesis

In [7]:

C4_model = read_sbml_model("C4_model.sbml")

In [9]:
# Setting photon uptake
BS_PPFD = 500
M_PPFD = 500
C4_model.reactions.Photon_tx1.upper_bound = M_PPFD
C4_model.reactions.Photon_tx3.upper_bound = BS_PPFD

#Defining maintenance cost in bundle sheath

BS_VATPase = 0.0049*BS_PPFD+2.7851
C4_model.reactions.get_by_id("ATPase_tx3").bounds = (BS_VATPase,BS_VATPase)

#Defining maintenance cost in Mesophyll

M_VATPase = 0.0049*M_PPFD+2.7851
C4_model.reactions.get_by_id("ATPase_tx1").bounds=(M_VATPase,M_VATPase)



In [10]:
# Restrict mesophyll sucrose, mesophyll starch and bundlesheath sucrose accumulation
for id in ["SUCROSE_v_dielTransfer_BS","SUCROSE_v_dielTransfer_M","STARCH_p_dielTransfer_M"]:
    rxn = C4_model.reactions.get_by_id(id)
    rxn.upper_bound = 0
    rxn.lower_bound = 0

In [11]:
C4sol = pfba(C4_model)
C4_model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
CARBON_DIOXIDE_e1,CO2_tx1,94.03,1,100.00%
WATER_e1,H2O_tx1,96.25,0,0.00%
NITRATE_e1,Nitrate_tx1,0.5877,0,0.00%
NITRATE_e4,Nitrate_tx4,0.3918,0,0.00%
OXYGEN_MOLECULE_e2,O2_tx2,2.787,0,0.00%
OXYGEN_MOLECULE_e4,O2_tx4,1.329,0,0.00%
Photon_e1,Photon_tx1,500,0,0.00%
Photon_e3,Photon_tx3,500,0,0.00%
SULFATE_e1,SO4_tx1,0.007489,0,0.00%
PROTON_c1,unlProtHYPO_c1,0.853,0,0.00%


In [20]:
for rxn in C4_model.reactions.query("_MBStransfer_Day"):
  if abs(rxn.flux)>20:
    print(rxn.id+"\t"+rxn.reaction+"\t"+str(rxn.flux))

GAP_c_MBStransfer_Day	GAP_c1 <=> GAP_c3	77.97396296998339
MAL_c_MBStransfer_Day	MAL_c1 <=> MAL_c3	39.645652981231855
G3P_c_MBStransfer_Day	G3P_c1 <=> G3P_c3	-78.11796842374217
Pi_c_MBStransfer_Day	Pi_c1 <=> Pi_c3	38.81087010676192
L_ALPHA_ALANINE_c_MBStransfer_Day	L_ALPHA_ALANINE_c1 <=> L_ALPHA_ALANINE_c3	-48.764133120049536
PHOSPHO_ENOL_PYRUVATE_c_MBStransfer_Day	PHOSPHO_ENOL_PYRUVATE_c1 <=> PHOSPHO_ENOL_PYRUVATE_c3	-55.30009469875812
L_ASPARTATE_c_MBStransfer_Day	L_ASPARTATE_c1 <=> L_ASPARTATE_c3	54.29496475206475


### When observing fluxes between mesophyll(M) and bundle sheath(BS) during the day, we see Aspartate, malate and triose-P transfer from M to BS. ALA, PEP and 3PG can also be observed to move from BS to M, indicative of C4 PEPCK.